# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,belle chasse,29.8549,-89.9906,31.99,72,40,4.12,US,1691635856
1,1,bubaque,11.2833,-15.8333,26.28,83,100,0.94,GW,1691635856
2,2,bilibino,68.0546,166.4372,16.84,42,96,3.57,RU,1691635856
3,3,bodaybo,57.8506,114.1933,25.19,55,100,1.17,RU,1691635857
4,4,tiksi,71.6872,128.8694,21.07,39,29,2.46,RU,1691635857


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.00,
    color = "City"
)

# Display the map
city_humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_conditions_df = city_data_df[(city_data_df['Max Temp'] <= 27) & (city_data_df['Max Temp'] >= 21) & (city_data_df['Wind Speed'] <= 4.5) & (city_data_df['Cloudiness'] == 0)] 

# Drop any rows with null values
ideal_weather_conditions_df.dropna()

# Display sample data
ideal_weather_conditions_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,zabol,32.2500,67.2500,26.90,13,0,3.50,AF,1691635861
34,34,douglas,41.3000,-96.1670,23.66,83,0,0.00,US,1691635867
59,59,bethel,41.3712,-73.4140,23.60,75,0,3.60,US,1691635742
97,97,kill devil hills,36.0307,-75.6760,26.66,82,0,2.06,US,1691635889
119,119,mashhad,36.2970,59.6062,24.08,43,0,2.06,IR,1691635803
123,123,santa cruz,-17.8000,-63.1667,23.70,73,0,3.60,BO,1691635798
175,175,al bawiti,28.3492,28.8659,23.98,62,0,2.81,EG,1691635915
208,208,mersa matruh,31.3525,27.2453,25.11,78,0,2.57,EG,1691635927
226,226,sao francisco,-15.9486,-44.8644,22.07,52,0,4.19,BR,1691635933
238,238,tadmur,34.5608,38.2805,25.97,56,0,3.07,SY,1691635937


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# https://www.statology.org/pandas-create-dataframe-from-existing-dataframe/
hotel_df = ideal_weather_conditions_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,zabol,AF,32.2500,67.2500,13,
34,douglas,US,41.3000,-96.1670,83,
59,bethel,US,41.3712,-73.4140,75,
97,kill devil hills,US,36.0307,-75.6760,82,
119,mashhad,IR,36.2970,59.6062,43,
123,santa cruz,BO,-17.8000,-63.1667,73,
175,al bawiti,EG,28.3492,28.8659,62,
208,mersa matruh,EG,31.3525,27.2453,78,
226,sao francisco,BR,-15.9486,-44.8644,52,
238,tadmur,SY,34.5608,38.2805,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    type = row['City']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

#Display sample data
hotel_df

Starting hotel search
zabol - nearest hotel: No hotel found
douglas - nearest hotel: Residence Inn Omaha West
bethel - nearest hotel: Hampton Inn Danbury
kill devil hills - nearest hotel: Mariner Days Inn & Suites
mashhad - nearest hotel: هتل تعاون
santa cruz - nearest hotel: Aviador Hotel Boutique
al bawiti - nearest hotel: Old Oasis Hotel
mersa matruh - nearest hotel: صقور
sao francisco - nearest hotel: Hotel Atalaia
tadmur - nearest hotel: فندق فيلا بالميرا
gremda - nearest hotel: Business Hotel Sfax
corinth - nearest hotel: Apollon
grand gaube - nearest hotel: Hotel Lux- Grand Gaube
bir el ater - nearest hotel: فندق الشفيق
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
hun - nearest hotel: فندق الهروج
susanville - nearest hotel: SureStay Plus Hotel By Best Western Susanville
divrigi - nearest hotel: No hotel found
birigui - nearest hotel: Birigui Palace Hotel
brookings - nearest hotel: Quality Inn
quchan - nearest hotel: هتل خیام
brigantine - nearest hotel: The S

,City,Country,Lat,Lng,Humidity,Hotel Name
13,zabol,AF,32.2500,67.2500,13,No hotel found
34,douglas,US,41.3000,-96.1670,83,Residence Inn Omaha West
59,bethel,US,41.3712,-73.4140,75,Hampton Inn Danbury
97,kill devil hills,US,36.0307,-75.6760,82,Mariner Days Inn & Suites
119,mashhad,IR,36.2970,59.6062,43,هتل تعاون
123,santa cruz,BO,-17.8000,-63.1667,73,Aviador Hotel Boutique
175,al bawiti,EG,28.3492,28.8659,62,Old Oasis Hotel
208,mersa matruh,EG,31.3525,27.2453,78,صقور
226,sao francisco,BR,-15.9486,-44.8644,52,Hotel Atalaia
238,tadmur,SY,34.5608,38.2805,56,فندق فيلا بالميرا


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1.25,
    color = "City",
    # Hover over places and gives you info on the hotel and country resource use: https://stackoverflow.com/questions/59678780/show-extra-columns-when-hovering-in-a-scatter-plot-with-hvplot
    hover_cols=['Country', 'Hotel Name']
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)